In [103]:
import sys
import warnings

warnings.filterwarnings('ignore')
sys.path.append(r"C:\Users\13477\Desktop\New Adventure\Goodreads\goodreads_prod")

from static import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException



def process_raw_books(books):
    res = books.drop_duplicates(subset = [col for col in books.columns if col != 'genres'])
    res = res.dropna()
    return res

def process_raw_reviews(reviews):
    """ Make sure each user only reviews a book once """
    res = reviews.drop_duplicates(subset = ['title', 'user_id'], keep = 'first')
    res = res.dropna()

    return res

# process dfs
original_books = pd.read_parquet("original_books.parquet")
original_books = process_raw_books(original_books)

original_reviews = pd.read_parquet("user_book_reviews.parquet")
original_reviews = process_raw_reviews(original_reviews)

new_books = pd.read_parquet("new_books.parquet")
new_books = process_raw_books(new_books)

new_reviews = pd.read_parquet("user_new_book_reviews.parquet")
new_reviews = process_raw_reviews(new_reviews)

original_users_top_5_reviews_full = pd.read_parquet("top_5_reviews_for_each_original_user.parquet")
original_users_top_5_reviews_full = process_raw_reviews(original_users_top_5_reviews_full)
original_users_top_5_reviews = original_users_top_5_reviews_full[['title', 'user_id', 'rating']]

In [27]:
# 29,636 books
all_books = pd.concat([original_books, new_books])
all_books = process_raw_books(all_books)

In [138]:
# 702,390 reviews (around 24 reviews per book)
all_reviews = pd.concat([original_reviews, new_reviews, original_users_top_5_reviews])
all_reviews = process_raw_reviews(all_reviews)

In [137]:
# 219,300 unique users
num_unique_users = all_reviews.user_id.nunique()

In [139]:
200000/30000

6.666666666666667

In [183]:
# 9466 users with data
users_data = pd.read_parquet("users_data.parquet")
users_data = users_data[users_data.num_ratings > 0]
users_with_data = users_data.user_id.unique()

In [130]:
users_data.is_best_reviewer.value_counts(normalize = True)

is_best_reviewer
False    0.811113
True     0.188887
Name: proportion, dtype: float64

In [131]:
users_data.is_most_followed.value_counts(normalize = True)

is_most_followed
False    0.915698
True     0.084302
Name: proportion, dtype: float64

#### Label genres for each book/review

In [39]:
target_genres = genres

mlb = MultiLabelBinarizer(classes=target_genres)
genre_matrix = mlb.fit_transform(all_books['genres'])

# Create a DataFrame with the new columns
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_, index=all_books.index)
genre_labels = pd.concat([all_books[['title']], genre_df], axis=1)

,title,Art,Biography,Business,Chick Lit,Children's,Christian,Classics,Comics,Contemporary,...,Romance,Science,Science Fiction,Self Help,Suspense,Spirituality,Sports,Thriller,Travel,Young Adult
0,Ways of Seeing,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Story of Art,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
all_labeled_reviews = all_reviews.merge(
        genre_labels, 
        on='title', 
        how='inner'
    )

all_labeled_reviews = all_labeled_reviews.drop_duplicates(subset=['title', 'user_id', 'rating'])

,title,user_id,rating,Art,Biography,Business,Chick Lit,Children's,Christian,Classics,...,Romance,Science,Science Fiction,Self Help,Suspense,Spirituality,Sports,Thriller,Travel,Young Adult
0,Ways of Seeing,175635-trevor,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ways of Seeing,2184529-justin-evans,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ways of Seeing,614778-ahmad-sharabiani,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Ways of Seeing,239699-pierce,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Ways of Seeing,64126678-casey,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907315,Live No Lies: Recognize and Resist the Three E...,123832897-brock-wiebe,4.0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
907316,The Anti-Christ,123832897-brock-wiebe,2.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
907317,One Day in the Life of Ivan Denisovich,1310936-jeremy,4.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
907318,The Shack,5885995-eric-molicki,1.0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


### Genre expertise score for each user

In [181]:
# num reviews of each user by genre
user_genre_counts = all_labeled_reviews.groupby('user_id')[genres].sum().T  # genres as index

# num reviews by each user
num_reviews_by_user = all_labeled_reviews.groupby('user_id')['title'].count()

# pct reviews by genre for each user
user_genre_pct = user_genre_counts.div(num_reviews_by_user, axis = 1)
user_genre_counts.head()

user_id,1-otis-chandler,10000084-heidi-clark,100001132-nikoleta,10000169-allegra,100002084-jacob-rutledge,100002600-shatha-awwad,1000041-smaileh,10000471-ellen-s-reviews,10000560-jacquelyn,100006355-torri-blackromanceconnoisseur,...,9999244-jessi-galloway,999928-kevin,99993439-philip,9999401-elizabeth,9999468-deanna,9999500-lexi,9999557-erika,9999943-gretchen-miller,9999971-cindy,9999999-mary
Art,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Biography,22,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
Business,34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Chick Lit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Children's,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [192]:
is_notable_user = users_data[['user_id', 'is_best_reviewer', 'is_most_followed']]
is_notable_user['is_notable_user'] = is_notable_user['is_best_reviewer'] | is_notable_user['is_most_followed']
is_notable_user

,user_id,is_best_reviewer,is_most_followed,is_notable_user
1,7869133-sarah,False,False,False
2,143499596-camden-glenn,False,False,False
3,83361162-josiah-edwards,False,False,False
4,63671935-gabriela-bevenuto,False,False,False
6,139483609-harry-taylor,False,False,False
...,...,...,...,...
14612,5856801-jimmy,False,False,False
14614,123832897-brock-wiebe,False,False,False
14615,1310936-jeremy,False,False,False
14617,5885995-eric-molicki,False,False,False


## Genre:Philosophy!!

### Ranking reviewers

In [214]:
my_genre = 'Philosophy'

genre_review_count_ranked = user_genre_counts.loc[my_genre, :].sort_values(ascending = False)
genre_review_count_ranked

user_id
614778-ahmad-sharabiani                    533
117399210-lu-s                             276
70395042-fergus-weaver-of-autistic-webs    198
175635-trevor                              190
29002534-valeriu-gherghel                  186
                                          ... 
20821048-almas-shamim                        0
20821223-karma                               0
2082127-ann                                  0
20821367-raquel                              0
9999999-mary                                 0
Name: Philosophy, Length: 219300, dtype: int32

In [215]:
genre_pct_of_reviews_ranked = user_genre_pct.loc[my_genre, :].sort_values(ascending = False)
genre_pct_of_reviews_ranked

user_id
55146818-luvcie          1.0
22772340-jenn            1.0
65299-anna               1.0
6530555-amanda           1.0
6530592-alina            1.0
                        ... 
20821048-almas-shamim    0.0
20821223-karma           0.0
2082127-ann              0.0
20821367-raquel          0.0
9999999-mary             0.0
Name: Philosophy, Length: 219300, dtype: float64

In [237]:
genre_ranker = pd.DataFrame({"review_count": genre_review_count_ranked, "review_pct": genre_pct_of_reviews_ranked})
genre_ranker['score1'] = genre_ranker['review_count'] * genre_ranker['review_pct']**1.2
genre_ranker = genre_ranker.sort_values(by = 'score1', ascending = False)

genre_ranker[genre_ranker.review_count >= 3]

,review_count,review_pct,score1
user_id,,,
6243880-beauregard-bottomley,113,0.748344,79.799513
175635-trevor,190,0.470297,76.842287
1651956-riku-sayuj,164,0.501529,71.647190
25683251,186,0.447115,70.797265
614778-ahmad-sharabiani,533,0.180007,68.088501
...,...,...,...
2752994-dem,3,0.010345,0.012439
26560207-gabby,3,0.009740,0.011572
17438949-melissa-dog-wolf-lover-martin,6,0.005305,0.011163


In [249]:
def get_top_n_reviewers(genre_ranker, n):
    top_n = genre_ranker.head(n)
    top_n['score1_normed'] = top_n['score1']/np.sum(top_n['score1'])

    return top_n

In [234]:
top_20 = genre_ranker.head(20)
top_20['score1_normed'] = top_20['score1']/np.sum(top_20['score1'])
top_20.head(5)

,review_count,review_pct,score1,score1_normed
user_id,,,,
6243880-beauregard-bottomley,113,0.748344,79.799513,0.077348
175635-trevor,190,0.470297,76.842287,0.074481
1651956-riku-sayuj,164,0.501529,71.647190,0.069446
25683251,186,0.447115,70.797265,0.068622
614778-ahmad-sharabiani,533,0.180007,68.088501,0.065996


### User item matrix

In [104]:
def construct_user_item_matrix(reviews):
    reviews_grouped = reviews.groupby(['user_id', 'title'])['rating'].mean().reset_index()
    user_item_matrix = reviews_grouped.pivot(index='user_id', columns='title', values='rating')
    user_item_matrix = user_item_matrix.fillna(0)

    return user_item_matrix

def filter_reviews_for_genre(all_reviews):
    # Merge genre labels with reviews on 'title'
    all_reviews_for_genre = all_reviews.merge(
        genre_labels[['title', my_genre]], 
        on='title', 
        how='inner'
    )
    
    all_reviews_for_genre = all_reviews_for_genre.drop_duplicates(subset=['title', 'user_id', 'rating'])
    all_reviews_for_genre = all_reviews_for_genre[all_reviews_for_genre[my_genre] == 1]
    all_reviews_for_genre = all_reviews_for_genre.drop(columns = my_genre)
    
    # only keep titles with at least 1 english char
    all_reviews_for_genre = all_reviews_for_genre[all_reviews_for_genre['title'].str.contains(r'[a-zA-Z]', regex=True)]

    return all_reviews_for_genre

# Function to safely detect language
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

def filter_english_reviews(reviews):
    all_titles = reviews.title.unique()
    english_titles = [t for t in all_titles if is_english(t)]

    english_reviews = reviews[reviews.title.isin(english_titles)]
    return english_reviews

In [105]:
all_reviews_for_genre = filter_reviews_for_genre(all_reviews)
genre_review = filter_english_reviews(all_reviews_for_genre)
genre_review

,title,user_id,rating
0,Ways of Seeing,175635-trevor,5.0
2,Ways of Seeing,2184529-justin-evans,3.0
4,Ways of Seeing,614778-ahmad-sharabiani,4.0
6,Ways of Seeing,239699-pierce,4.0
8,Ways of Seeing,64126678-casey,3.0
...,...,...,...
907199,The Happiness Hypothesis: Finding Modern Truth...,2700944-corey-decker,0.0
907224,The Problem of Pain,1664399-maureen-garza,5.0
907302,Antifragile: Things That Gain from Disorder,5069466-paul-mullen,3.0
907308,Mere Christianity,20102257-jeremiah-lorrig,5.0


In [275]:
def condense_user_item_matrix(user_item_matrix, n = 3):

    filtered_matrix = user_item_matrix[user_item_matrix.astype(bool).sum(axis=1) >= n]
    filtered_matrix = filtered_matrix.loc[:, filtered_matrix.astype(bool).sum(axis=0) >= n]

    return filtered_matrix

In [308]:
user_item_matrix = construct_user_item_matrix(genre_review)
condensed_user_item_matrix = condense_user_item_matrix(user_item_matrix)
condensed_user_item_matrix

title,#ACCELERATE: Manifesto for an Accelerationist Politics,10 Books That Screwed Up the World: And 5 Others That Didn't Help,101 Essays That Will Change The Way You Think,177 Mental Toughness Secrets of the World Class,21 Lessons for the 21st Century,36 Arguments for the Existence of God: A Work of Fiction,"50 Psychology Classics: Who We Are, How We Think, What We Do: Insight and Inspiration from 50 Key Books",7 Strategies for Wealth & Happiness: Power Ideas from America's Foremost Business Philosopher,"80,000 Hours: Find a fulfilling career that does good",A Book Forged in Hell: Spinoza's Scandalous Treatise and the Birth of the Secular Age,...,Yuganta: The End of an Epoch,"Zen Mind, Beginner's Mind: Informal Talks on Zen Meditation and Practice",Zen and the Art of Motorcycle Maintenance,Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zen and the Psychology of Transformation: The Supreme Doctrine,Zero: The Biography of a Dangerous Idea,Zhuan Falun: The Complete Teachings of Falun Gong,Zorba the Greek,the prophet
user_id,,,,,,,,,,,,,,,,,,,,,
1-otis-chandler,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10023481-peter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100282-colleen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003057-myridian,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003139-rhyd-wildermuth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962738-my-name-is-corey-irl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99747-chris,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99766625-brok3n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Expert suggested books

In [378]:
top_50 = get_top_n_reviewers(genre_ranker, 50)
amount_of_say = top_50[['score1_normed']]

In [379]:
# ratings by the "experts"
expert_user_item_matrix = condensed_user_item_matrix[condensed_user_item_matrix.index.isin(top_50.index)]
expert_user_item_matrix = expert_user_item_matrix.loc[top_50.index]

# scored by experts
expert_ratings = expert_user_item_matrix.T.dot(amount_of_say)
expert_ratings.columns = ['expert_score']
expert_ratings = expert_ratings.drop_duplicates()
expert_ratings = expert_ratings.sort_values(by = 'expert_score', ascending = False)
expert_ratings.head(10)

,expert_score
title,
The Symposium,1.522988
"In Search of Lost Time, Volume 1: The Way by Swann's",1.339018
The Republic,1.335848
Swann's Way,1.281911
The Trial,1.265848
A History of Western Philosophy,1.228953
A Brief History of Time,1.163613
The Metamorphosis,1.121083
Thus Spoke Zarathustra,1.116125


In [380]:
expert_user_item_matrix

title,#ACCELERATE: Manifesto for an Accelerationist Politics,10 Books That Screwed Up the World: And 5 Others That Didn't Help,101 Essays That Will Change The Way You Think,177 Mental Toughness Secrets of the World Class,21 Lessons for the 21st Century,36 Arguments for the Existence of God: A Work of Fiction,"50 Psychology Classics: Who We Are, How We Think, What We Do: Insight and Inspiration from 50 Key Books",7 Strategies for Wealth & Happiness: Power Ideas from America's Foremost Business Philosopher,"80,000 Hours: Find a fulfilling career that does good",A Book Forged in Hell: Spinoza's Scandalous Treatise and the Birth of the Secular Age,...,Yuganta: The End of an Epoch,"Zen Mind, Beginner's Mind: Informal Talks on Zen Meditation and Practice",Zen and the Art of Motorcycle Maintenance,Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zen and the Psychology of Transformation: The Supreme Doctrine,Zero: The Biography of a Dangerous Idea,Zhuan Falun: The Complete Teachings of Falun Gong,Zorba the Greek,the prophet
user_id,,,,,,,,,,,,,,,,,,,,,
6243880-beauregard-bottomley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175635-trevor,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1651956-riku-sayuj,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0
25683251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
614778-ahmad-sharabiani,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,5.0
70395042-fergus-weaver-of-autistic-webs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0
60074558-xander,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42133960-peiman-e-iran,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117399210-lu-s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [381]:
adjusted_expert_user_item_matrix = expert_user_item_matrix.where(expert_user_item_matrix == 0, expert_user_item_matrix - 3)

In [382]:
# scored by experts
adjusted_expert_ratings = adjusted_expert_user_item_matrix.T.dot(amount_of_say)
adjusted_expert_ratings.columns = ['expert_score']
adjusted_expert_ratings = adjusted_expert_ratings.drop_duplicates()
adjusted_expert_ratings = adjusted_expert_ratings.sort_values(by = 'expert_score', ascending = False)
adjusted_expert_ratings.head(10)

,expert_score
title,
The Symposium,0.525583
"In Search of Lost Time, Volume 1: The Way by Swann's",0.510882
Swann's Way,0.488039
The Trial,0.446276
The Republic,0.395108
The Metamorphosis,0.385756
Critique of Pure Reason,0.374642
Thus Spoke Zarathustra,0.371209
Phenomenology of Spirit,0.365706


In [413]:
def lookup_rating(user_item_matrix, user_id, book_name):
    return user_item_matrix.loc[user_id, book_name]

def how_many_read(book_name, experts):
    wavgs = amount_of_say.copy()
    wavgs['book_rating'] = [lookup_rating(expert_user_item_matrix, u, book_name) for u in experts]
    wavgs = wavgs[wavgs.book_rating != 0]

def avg_expert_rating(book_name, experts):
    wavgs = amount_of_say.copy()
    wavgs['book_rating'] = [lookup_rating(expert_user_item_matrix, u, book_name) for u in experts]
    wavgs = wavgs[wavgs.book_rating != 0]

    res = np.dot(wavgs['score1_normed'], wavgs['book_rating'])/np.sum(wavgs['score1_normed'])
    
    return res, len(wavgs)

In [414]:
adjusted_expert_ratings[['wavg_rating','experts_who_read']] = [avg_expert_rating(book, experts) for book in adjusted_expert_ratings.index]
adjusted_expert_ratings.head(10)

,expert_score,wavg_rating,experts_who_read
title,,,
The Symposium,0.525583,4.580849,11.0
"In Search of Lost Time, Volume 1: The Way by Swann's",0.510882,4.850718,10.0
Swann's Way,0.488039,4.844275,9.0
The Trial,0.446276,4.633572,10.0
The Republic,0.395108,4.259992,10.0
The Metamorphosis,0.385756,4.573816,10.0
Critique of Pure Reason,0.374642,4.920568,7.0
Thus Spoke Zarathustra,0.371209,4.494971,9.0
Phenomenology of Spirit,0.365706,4.722839,7.0
